In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# stuff needed for google colab
CURR_DIR = "/content/drive/My Drive/Grad School Fall/CS 573/Project/colab_notebooks"
DATA_FILE = "profiles.csv"

In [4]:
import pandas as pd
import numpy as np
import os
import nltk
import sys
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from scipy import sparse
import string

np.set_printoptions(threshold=1000)

In [5]:
profiles = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
print(profiles)

       age  ...     status
0       22  ...     single
1       35  ...     single
2       38  ...  available
3       23  ...     single
4       29  ...     single
...    ...  ...        ...
59941   59  ...     single
59942   24  ...     single
59943   42  ...     single
59944   27  ...     single
59945   39  ...     single

[59946 rows x 31 columns]


In [6]:
train_df = profiles.sample(10000, random_state=42)

In [7]:
def parseEssay(df):
  essay_list = df['essay0'].to_list()

  essay_list1 = df['essay1'].to_list()
  essay_list2 = df['essay2'].to_list()

  # parses out the NaN's and leaves them empty
  # jesus christ look at this garbage code
  print("parsing out nans")
  for i in range(len(essay_list)):
    if not isinstance(essay_list[i], str):
      if not isinstance(essay_list1[i], str):
        if not isinstance(essay_list1[i], str):
          essay_list[i] = ""
        else:
          essay_list[i] = essay_list2[i]
      else:
        essay_list[i] = essay_list1[i]

  return essay_list

preclean_train_essays = parseEssay(train_df)

parsing out nans


In [8]:
table = str.maketrans(dict.fromkeys(string.punctuation))
# keep the ' so that we can have words like "i'm", "can't", etc
del table[39]

# print(table)

def cleanLine(line, punc_table):
  # remove <br />
  # print(line)
  line = line.replace("<br />", " ")
  line = line.replace("\n", " ")
  line = line.lower()
  line = line.translate(punc_table)
  line = ' '.join(line.split())

  return line


def cleanEssays(data):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  for i in range(len(data)):
    data[i] = cleanLine(data[i], table)
    # break

  return data

# print(train_essays[4])
train_essays = cleanEssays(preclean_train_essays)
# cleanLine(train_essays[4], table)
print(train_essays[4])
# print(len(train_essays[999].split()))


i come to san francisco by way of new york where i was born raised and went to school i think this accounts for my occasionally blunt no bullshit attitude although the bay area has been hard at work mellowing me out some say its even been successful ive been working in the games industry in the east bay as a producer designer and occasional contributor of polygonal art for the past six years recent projects involved me working with a bunch of neat folks to make the games where you click buttons on plastic guitar thingies and pretend to play music it was great fun working with my team of bright super passionate music loving people and i learned a lot but the 1416 work days eventually took their toll so now i work saner hours making educational games for 38 yearolds in emeryville its humbling really to see all your design assumptions blown out of the water by a 3 year old im a big fan of city walking previous to coming to sf i worked in new york a city of walkers and drivers for that mat

In [9]:
# def parseEssay(df):
#   essay_list = df['essay0'].to_list()

#   # parses out the NaN's and leaves them empty
#   print("parsing out nans")
#   for i in range(len(essay_list)):
#     if not isinstance(essay_list[i], str):
#       essay_list[i] = ""

#   return essay_list

MAX_WORDS = 39170
MAX_LEN = 300
def createWordVectors(train_essays, max_words, max_len):
  print("encoding essays")
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(train_essays)
  sequences = tok.texts_to_sequences(train_essays)
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

  # word_vecs = vectorizer.transform(train_essays)

  return sequences_matrix

# train_essays = parseEssay(train_df)
# word_vecs, vectorizer = createWordVectors(train_essays)
word_vecs = createWordVectors(train_essays, MAX_WORDS, MAX_LEN)

X = word_vecs

# for printing out the actual vocab encoding vec in case you want it
# print(vectorizer.vocabulary_)
# encoding document
print(X.shape)


encoding essays
(10000, 300)


In [10]:
print(X[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [11]:
def load_labels(df):
  df['sex'].mask(df['sex'] == 'm', 0, inplace=True)
  df['sex'].mask(df['sex'] == 'f', 1, inplace=True)

  labels = df['sex'].to_numpy()

  return labels

labels = load_labels(train_df)

# Convert to float32's so that they play nicely

In [12]:
X = np.asarray(X).astype('float32')
Y = np.asarray(labels).astype('float32')

# Do some neural net shenannigans

In [24]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import KFold
import statistics 

In [27]:
def createModel(max_words, max_len):
  model = Sequential([
    Embedding(max_words, 64,input_length=max_len),
    Bidirectional(LSTM(64,  return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(12, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4),
              metrics=['accuracy'])

  return model

In [28]:
n_split = 10

best_acc = 0
best_model = None

accuracies = []

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test = X[train_index],X[test_index]
  y_train,y_test = Y[train_index],Y[test_index]

  # print(x_train.shape)
  # print(y_train.shape)
  model = createModel(MAX_WORDS, MAX_LEN)

  model.fit(x_train, y_train,epochs=10)
  
  _, test_acc = model.evaluate(x_test,y_test)
  
  accuracies.append(test_acc)

  # break

avg_acc = statistics.mean(accuracies)
print('Test set accuracy: %.2f' % avg_acc)
print(accuracies)

Epoch 1/10
282/282 [==============================] - 17s 59ms/step - loss: 0.6812 - accuracy: 0.5847
Epoch 2/10
282/282 [==============================] - 17s 59ms/step - loss: 0.6760 - accuracy: 0.5891
Epoch 3/10
282/282 [==============================] - 17s 59ms/step - loss: 0.6493 - accuracy: 0.6303
Epoch 4/10
282/282 [==============================] - 17s 59ms/step - loss: 0.5057 - accuracy: 0.7556
Epoch 5/10
282/282 [==============================] - 16s 58ms/step - loss: 0.3494 - accuracy: 0.8444
Epoch 6/10
282/282 [==============================] - 17s 59ms/step - loss: 0.2431 - accuracy: 0.8966
Epoch 7/10
282/282 [==============================] - 17s 59ms/step - loss: 0.1745 - accuracy: 0.9280
Epoch 8/10
282/282 [==============================] - 17s 59ms/step - loss: 0.1386 - accuracy: 0.9396
Epoch 9/10
282/282 [==============================] - 17s 59ms/step - loss: 0.1146 - accuracy: 0.9510
Epoch 10/10
32/32 [==============================] - 1s 21ms/step - loss: 1.4124 -

In [ ]:
print(accuracies)